In [1]:
import pandas as pd
import numpy as np
import sqlite3
conn = sqlite3.connect('webshop.sqlite')

In [2]:
tables_df = pd.read_sql_query(
    "SELECT name AS table_name FROM sqlite_master WHERE type='table';",
    conn
)

tables_df

,table_name
0,dates
1,customer
2,product
3,country
4,webshop_sales
5,browser
6,webshop_visits


In [4]:
df1 = pd.read_sql_query("SELECT * FROM customer", conn)

# view data
df1.head()

,id,name
0,1,Parker Group
1,2,"Heidenreich, Baumbach and Carter"
2,3,"Ondricka, Klein and Larkin"
3,4,"Berge, Tillman and Roberts"
4,5,Steuber Ltd


In [5]:
df2 = pd.read_sql_query("SELECT * FROM product", conn)

# view data
df2.head()

,id,category_label,product_label
0,1,Books,Is there a limit to productivity or can it rai...
1,2,Supplies,Gel Mouse Pad
2,3,Office electronics,Air Conditioner ClimaTronic
3,4,Supplies,Gun Cylinder Pen Holder
4,5,Supplies,Portable laptop stand/table


In [6]:
df = df1.merge(df2, how="cross")
df.head()

,id_x,name,id_y,category_label,product_label
0,1,Parker Group,1,Books,Is there a limit to productivity or can it rai...
1,1,Parker Group,2,Supplies,Gel Mouse Pad
2,1,Parker Group,3,Office electronics,Air Conditioner ClimaTronic
3,1,Parker Group,4,Supplies,Gun Cylinder Pen Holder
4,1,Parker Group,5,Supplies,Portable laptop stand/table


In [7]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("webshop.sqlite")

sales = pd.read_sql_query("SELECT * FROM webshop_sales", conn)
product = pd.read_sql_query("SELECT * FROM product", conn)




In [10]:
sales.to_csv("sales.csv", index=False)

In [8]:
sales.head()

,id,date_sale_id,customer_id,product_id,country_id,quantity,price_total,delivery_days
0,1,2012_07_28,1,1,mexico,0,0.00,2
1,2,2014_07_03,2,2,germany,4,27.80,13
2,3,2013_07_16,3,3,tanzania,0,0.00,9
3,4,2016_02_09,3,4,egypt,0,0.00,10
4,5,2012_11_27,4,5,united_states,5,199.85,6


In [17]:
product.head()

,id,category_label,product_label
0,1,Books,Is there a limit to productivity or can it rai...
1,2,Supplies,Gel Mouse Pad
2,3,Office electronics,Air Conditioner ClimaTronic
3,4,Supplies,Gun Cylinder Pen Holder
4,5,Supplies,Portable laptop stand/table


In [19]:
df = sales.merge(
    product,
    left_on="product_id",
    right_on="id",
    how="inner"
)

df.head()


,id_x,date_sale_id,customer_id,product_id,country_id,quantity,price_total,delivery_days,id_y,category_label,product_label
0,1,2012_07_28,1,1,mexico,0,0.00,2,1,Books,Is there a limit to productivity or can it rai...
1,2,2014_07_03,2,2,germany,4,27.80,13,2,Supplies,Gel Mouse Pad
2,3,2013_07_16,3,3,tanzania,0,0.00,9,3,Office electronics,Air Conditioner ClimaTronic
3,4,2016_02_09,3,4,egypt,0,0.00,10,4,Supplies,Gun Cylinder Pen Holder
4,5,2012_11_27,4,5,united_states,5,199.85,6,5,Supplies,Portable laptop stand/table


# OLAP OPERATIONS

# ROLL-UP

In [21]:
df.groupby("category_label")["price_total"].sum()

category_label
Books                 101684.69
Furniture              42707.29
Office basics          14818.08
Office electronics    394458.93
Paper & Stationery     24560.27
Software                   0.00
Supplies               20623.51
Writing                 3695.03
Name: price_total, dtype: float64

# DRILL-DOWN

In [22]:
df.groupby(
    ["category_label", "product_label"]
)["price_total"].sum()


category_label      product_label                                                      
Books               Copying with your boss daiy (2013)                                       5535.00
                    Finding a good self-help book (2014)                                     3208.50
                    Getting StackOvercrowded users to code for you (2013)                    8717.00
                    Is there a limit to productivity or can it raise to infiniy? (2012)     11027.50
                    Mastering the fax machine (2012)                                         4377.81
                    Proper whiteboard erasing (2008)                                         2610.96
                    Rise of AI and the end of economy as we know it (2015)                  10636.20
                    Rolling your own cryptography system for best security (2015)            9447.50
                    Security through obscurity best practices (2011)                        11168.19
   

# SLICE

In [23]:
df[df["category_label"] == "Supplies"]


,id_x,date_sale_id,customer_id,product_id,country_id,quantity,price_total,delivery_days,id_y,category_label,product_label
1,2,2014_07_03,2,2,germany,4,27.80,13,2,Supplies,Gel Mouse Pad
3,4,2016_02_09,3,4,egypt,0,0.00,10,4,Supplies,Gun Cylinder Pen Holder
4,5,2012_11_27,4,5,united_states,5,199.85,6,5,Supplies,Portable laptop stand/table
9,10,2014_01_09,9,10,turkey,7,76.09,2,10,Supplies,Desk Supplies Organizer Caddy
28,29,2015_01_29,25,27,democratic_republic_of_the_congo,0,0.00,5,27,Supplies,Wrist-rest Support Contraption
...,...,...,...,...,...,...,...,...,...,...,...
4504,4505,2013_09_10,69,10,india,0,0.00,6,10,Supplies,Desk Supplies Organizer Caddy
4510,4511,2016_12_25,81,27,tanzania,3,58.32,6,27,Supplies,Wrist-rest Support Contraption
4521,4522,2013_09_10,29,2,nigeria,1,6.95,2,2,Supplies,Gel Mouse Pad
4540,4541,2016_12_16,200,2,brazil,1,6.95,6,2,Supplies,Gel Mouse Pad


# DICE

In [24]:
df[
    (df["category_label"] == "Supplies") &
    (df["country_id"] == "germany") &
    (df["price_total"] > 0)
]

,id_x,date_sale_id,customer_id,product_id,country_id,quantity,price_total,delivery_days,id_y,category_label,product_label
1,2,2014_07_03,2,2,germany,4,27.80,13,2,Supplies,Gel Mouse Pad
927,928,2012_03_06,144,10,germany,5,54.35,5,10,Supplies,Desk Supplies Organizer Caddy
1521,1522,2013_09_23,138,2,germany,3,20.85,9,2,Supplies,Gel Mouse Pad
1874,1875,2014_04_27,141,10,germany,1,10.87,10,10,Supplies,Desk Supplies Organizer Caddy
2196,2197,2013_12_30,200,2,germany,2,13.90,8,2,Supplies,Gel Mouse Pad
2273,2274,2015_10_11,32,4,germany,5,89.95,8,4,Supplies,Gun Cylinder Pen Holder
2625,2626,2015_10_15,73,2,germany,4,27.80,5,2,Supplies,Gel Mouse Pad
3008,3009,2015_08_10,160,10,germany,3,32.61,5,10,Supplies,Desk Supplies Organizer Caddy
3434,3435,2015_07_18,112,4,germany,3,53.97,6,4,Supplies,Gun Cylinder Pen Holder
3485,3486,2013_03_07,154,10,germany,1,10.87,4,10,Supplies,Desk Supplies Organizer Caddy


# PIVOT

In [25]:
pd.pivot_table(
    df,
    values="price_total",
    index="product_label",
    columns="country_id",
    aggfunc="sum",
    fill_value=0
)


country_id,argentina,bangladesh,brazil,china,colombia,democratic_republic_of_the_congo,egypt,ethiopia,france,germany,...,south_africa,south_korea,spain,tanzania,thailand,turkey,ukraine,united_kingdom,united_states,vietnam
product_label,,,,,,,,,,,,,,,,,,,,,
Air Conditioner ClimaTronic,799.98,0.00,3999.90,6799.83,3199.92,399.99,1999.95,0.00,1199.97,1199.97,...,0.00,2399.94,0.00,0.00,399.99,0.00,0.00,1999.95,2399.94,3999.90
BeholderTM Webcam (with motion detection),778.20,0.00,129.70,129.70,778.20,518.80,389.10,1037.60,259.40,518.80,...,1297.00,129.70,778.20,1556.40,259.40,648.50,259.40,648.50,259.40,778.20
Bleach whitened recycled paper (2000 sheets),119.97,79.98,239.94,839.79,319.92,0.00,39.99,159.96,79.98,0.00,...,279.93,559.86,279.93,359.91,199.95,0.00,0.00,319.92,279.93,199.95
Breeze Liner,14.97,54.89,24.95,99.80,84.83,54.89,44.91,54.89,39.92,9.98,...,19.96,59.88,24.95,29.94,44.91,34.93,14.97,19.96,34.93,49.90
Burn-O-Matic 3000,239.92,119.96,89.97,329.89,239.92,0.00,449.85,29.99,0.00,359.88,...,149.95,389.87,299.90,59.98,269.91,269.91,59.98,569.81,179.94,149.95
Classic Notebook,129.10,64.55,129.10,219.47,51.64,25.82,167.83,103.28,51.64,180.74,...,51.64,12.91,38.73,77.46,12.91,206.56,77.46,51.64,129.10,0.00
Copying with your boss daiy (2013),157.50,67.50,180.00,247.50,405.00,22.50,270.00,247.50,0.00,135.00,...,135.00,337.50,247.50,112.50,45.00,112.50,202.50,180.00,90.00,292.50
Cubes: Fantastic Python OLAP/ROLAP server,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
CubesViewer: Awesome HTML5 OLAP viewer for Cubes,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [26]:
df.groupby("category_label")["quantity"].sum()


category_label
Books                 2881
Furniture              671
Office basics          681
Office electronics    1847
Paper & Stationery    1733
Software               339
Supplies              1112
Writing                597
Name: quantity, dtype: int64

In [27]:
df.groupby("category_label")["delivery_days"].mean()


category_label
Books                 6.614686
Furniture             6.384824
Office basics         6.623288
Office electronics    6.450000
Paper & Stationery    6.661267
Software              6.314465
Supplies              6.439130
Writing               6.432432
Name: delivery_days, dtype: float64